In [6]:
from utils import *


In [2]:
DATE = 423

inputFile = '../data/居住地信息/' + str(DATE).rjust(4, '0') + '.txt'

allDistricts = ['浦东新区', '虹口区', '闵行区', '徐汇区', '黄浦区', '嘉定区', '静安区',
                '杨浦区', '宝山区', '奉贤区', '松江区', '崇明区', '青浦区', '金山区', '长宁区', '普陀区']


In [7]:
# 读取&整理inputFile，存入today: list中

print('parsing',inputFile)
today = []
# 3/9 ～ 3/17
'''
with open(inputFile) as f:
    for line in f:
        try:
            line = re.findall("居住于(.+)",line)[0]
        except:
            continue
        line = line.split('\n')[0]
        line = line.split('（')[0]
        line = line.split('(')[0]
        
        today.append(line)
'''


# 3/18及之后
currDistrict = ""
with open(inputFile) as f:
    for line in f:
        line = line.split('\n')[0]
        line = line.split('（')[0]
        line = line.split('(')[0]
        line = line.split('，')[0]
        
        if line in allDistricts:
            currDistrict = line
            continue
        today.append(currDistrict+line)


parsing ../data/居住地信息/0423.txt


UnicodeDecodeError: 'gbk' codec can't decode byte 0xaf in position 21: illegal multibyte sequence

In [ ]:
# 保存整理后得到的today

np.save("../data/居住地信息_processed/"+inputFile.split('/')[-1].split(sep='.')[0]+".npy", today)
print("save to", "../data/居住地信息_proacessed/"+inputFile.split('/')[-1].split(sep='.')[0]+".npy")

# 此处结果已储存至文件夹中

In [8]:
# 读取 all_address.npy

try:
    all_addresses = list(np.load('../data/居住地信息_processed/all_addresses.npy'))
    for i in range(len(all_addresses)):
        all_addresses[i] = all_addresses[i].split('（')[0]
    print("read from all_addresses.npy")
except:
    print("There's no all_addresses.npy. Initializing a new one...")
    all_addresses = []


read from all_addresses.npy


# 将新地址加入到 all_address 中，更新 all_address.npy

print("updating all_addresses.npy...")
prev = len(all_addresses)

all_addresses_new = all_addresses + today
all_addresses_new = sorted(list(set(all_addresses_new)))

new_address = list(set(all_addresses_new) - set(all_addresses))

for district in allDistricts:
    if district in all_addresses_new:
        all_addresses_new.remove(district)
        
curr = len(all_addresses_new)
assert len(new_address) == curr-prev
print("added", curr-prev, 'elements')
all_addresses = all_addresses_new
np.save('../data/居住地信息_processed/all_addresses.npy', all_addresses)
print('done.')


In [10]:
# 读取 estate_info.csv

try:
    estate_info = pd.read_csv('../data/daily_info/estate_info_'+str(DATE-1).rjust(4, '0')+'.csv')
    print('reading from', './data/estate_info_'+str(DATE-1).rjust(4, '0')+'.csv')

except:
    print('cannot find ../data/daily_info/estate_info_'+str(DATE).rjust(4, '0')+'.csv  Initializing a new estate_info...')
    estate_info = pd.DataFrame(columns=['address', 'lon', 'lat', 'name', 'poiType', 'city', 'district', 'town', 'encode'])


cannot find ../data/daily_info/estate_info_0423.csv  Initializing a new estate_info...


In [ ]:
# 根据address查询gps坐标(BD09)，然后根据gps坐标获取对应的POI名称，类型，所属区县，街道

# myAK = 'TnG2vjIajzrD4px4pyrod13ns2qTGPOu'
myAK = 'qW4FuG8qBLFMMuOP4Tht4xxnPpbNCRCY'

if len(new_address) > 0:
    new_estate_info = pd.DataFrame(new_address, columns=['address'])

    pandarallel.initialize(progress_bar=True, nb_workers=64)
    new_estate_info[['city', 'district', 'town', 'name', 'poiType', 'lon_wgs', 'lat_wgs','source', 'confidence']] = new_estate_info.parallel_apply(lambda x: GD_GPS_TownInfo(x.address), axis=1, result_type='expand')


    # pandarallel.initialize(progress_bar=True,nb_workers=15)
    # new_estate_info[['lon', 'lat']] = new_estate_info.parallel_apply(lambda x: getGPS(x['address'], myAK), axis=1, result_type='expand')
    # pandarallel.initialize(progress_bar=True,nb_workers=22)
    # new_estate_info[['lon_gd', 'lat_gd']] = new_estate_info.parallel_apply(lambda x: GD_GPS(x['address']), axis=1, result_type='expand')

    # pandarallel.initialize(progress_bar=True,nb_workers=15)
    # new_estate_info[['name', 'poiType', 'city', 'district', 'town']] = new_estate_info.parallel_apply(lambda x: getTownInfo(x['lon'], x['lat'], myAK), axis=1, result_type='expand')
    # pandarallel.initialize(progress_bar=True,nb_workers=22)
    # new_estate_info[['name_gd', 'city_gd', 'district_gd', 'town_gd']] = new_estate_info.parallel_apply(lambda x: GD_townInfo(x['lon_gd'], x['lat_gd']), axis=1, result_type='expand')

In [ ]:
# 将BD09坐标系转换成wgs

# estate_info[['lon_wgs', 'lat_wgs']] = estate_info.parallel_apply(lambda x: bd09_to_wgs84(x['lon'], x['lat']), axis=1, result_type='expand')

# if len(new_address) > 0:
#     pandarallel.initialize(progress_bar=False,nb_workers=60)
#     new_estate_info[['lon_wgs', 'lat_wgs']] = new_estate_info.parallel_apply(lambda x: bd09_to_wgs84(x['lon'], x['lat']), axis=1, result_type='expand')
#     new_estate_info.query("name == '-1'")


In [ ]:
# 合并new_estate_info，estate_info

if len(new_address) > 0:
    estate_info = pd.concat([estate_info, new_estate_info])
# estate_info

In [ ]:
# 生成 gps encoder/decoder dict

e_CNTs = []
gps_encoder = {}
gps_decoder = {}
d_CNT = 1
for d in allDistricts: # 区县编码
    gps_encoder[d] = [str(d_CNT).rjust(2, '0')]
    gps_decoder[str(d_CNT).rjust(2, '0')] = [d]
    # print(d)
    allTowns = estate_info.query('district == @d')['town'].unique()
    town_encoder = {}
    town_decoder = {}

    t_CNT = 1
    for t in allTowns: # 街道编码
        town_encoder[t] = [str(t_CNT).rjust(2, '0')]
        town_decoder[str(t_CNT).rjust(2, '0')] = [t]
        allGPS = list(set(zip(estate_info.query('town == @t').lon_wgs, estate_info.query('town == @t').lat_wgs)))
        estate_encode = {}
        estate_decode = {}

        e_CNT = 1
        for e in allGPS: # gps编码
            estate_encode[str(e)] = str(e_CNT).rjust(3, '0')
            estate_decode[str(e_CNT).rjust(3, '0')] = e
            e_CNT += 1
        town_encoder[t].append(estate_encode)
        town_decoder[str(t_CNT).rjust(2, '0')].append(estate_decode)

        # if e_CNT >= 500:
        #     print(t, e_CNT)

            
        e_CNTs.append(e_CNT)
        t_CNT += 1

    gps_encoder[d].append(town_encoder)
    gps_decoder[str(d_CNT).rjust(2, '0')].append(town_decoder)

    d_CNT += 1

with open('../data/daily_info/gps_encoder.json', 'w', encoding='utf-8') as fp:
    json.dump(gps_encoder, fp, ensure_ascii=False)
print('../data/daily_info/gps_encoder.json updated')

with open('../data/daily_info/gps_decoder.json', 'w', encoding='utf-8') as fp:
    json.dump(gps_decoder, fp, ensure_ascii=False)
print('../data/daily_info/gps_decoder.json updated')

# max(e_CNTs)


In [ ]:
# 根据gps_encoder添加编码信息
pandarallel.initialize(progress_bar=False,nb_workers=60)
estate_info['encode'] = estate_info.parallel_apply(lambda x : Encoder(x,gps_encoder), axis=1)

In [ ]:
# 去除gps相同的地址，生成每日estate_info.csv, estate_info_simp.csv

data = []
for lon_wgs, lat_wgs in tqdm(set(zip(estate_info.lon_wgs, estate_info.lat_wgs))):
    data.append(list(estate_info.query("lon_wgs == @lon_wgs & lat_wgs == @lat_wgs").iloc[0, :].values))

simp = pd.DataFrame(data, columns=estate_info.columns).sort_values('address')

simp.to_csv('../data/daily_info/estate_info_'+str(DATE).rjust(4, '0')+'_simp.csv', index=False, encoding='utf_8_sig')
print('../data/daily_info/estate_info_'+str(DATE).rjust(4, '0')+'_simp.csv generated')

estate_info.to_csv('../data/daily_info/estate_info_'+str(DATE).rjust(4, '0')+'.csv',index=False, encoding='utf_8_sig')
print('../data/daily_info/estate_info_'+str(DATE).rjust(4, '0')+'.csv generated')

address_encode = {}
address_decode = {}

for item in list(zip(estate_info.address, estate_info.encode)):
    a,e = item
    address_encode[a] = e

for item in list(zip(simp.address, simp.encode)):
    a, e = item
    address_decode[e] = a

with open('../data/daily_info/address_encode.json', 'w', encoding='utf-8') as fp:
    json.dump(address_encode, fp, ensure_ascii=False)
print("../data/daily_info/address_encode.json updated")

with open('../data/daily_info/address_decode.json', 'w', encoding='utf-8') as fp:
    json.dump(address_decode, fp, ensure_ascii=False)
print("../data/daily_info/address_decode.json updated")


In [ ]:
len(address_decode.keys())

In [ ]:
# 生成每日 time_series.csv

with open('../data/daily_info/address_decode.json') as fp:
    address_decode = json.load(fp)

with open('../data/daily_info/address_encode.json') as fp:
    address_encode = json.load(fp)

all_codes = address_decode.keys()

time_series = pd.DataFrame(index=all_codes)

file_dir = "../data/居住地信息_processed"
listDir=os.listdir(file_dir)
for fName in tqdm(listDir):
    rslt = re.findall('\d+\.npy', fName)
    if len(rslt) == 0:
        continue
    f = rslt[0]
    date = f.split('.')[0]
    today = list(np.load('../data/居住地信息_processed/' + f))
    time_series.loc[:,date] = 0


    for item in today:
        item = item.split('（')[0]
        if item in allDistricts:
            continue
        item = address_encode[item]
        
        time_series.loc[item,date] = 1



time_series = time_series[sorted(time_series.columns)].fillna(0)

time_series.to_csv('../data/daily_info/time_series_'+ str(DATE).rjust(4, '0')+'.csv')
print('../data/daily_info/time_series_'+str(DATE).rjust(4, '0')+'.csv generated')

time_series.to_csv('../data/CaseInfo_April/time_series.csv')
print('../data/CaseInfo_April/time_series.csv generated')

In [ ]:
# 更新 estate_info_POI_category.json

out_geojson_path = '../data/CaseInfo_April/estate_info_POI_category.json'

features = []
for row in simp.itertuples():
    address = getattr(row, "address")
    lon = getattr(row, "lon_wgs")
    lat = getattr(row, "lat_wgs")
    name = getattr(row, "name")
    poiType = getattr(row, "poiType")
    city = getattr(row, "city")
    district = getattr(row, "district")
    town = getattr(row, "town")
    encode = getattr(row, "encode")

    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [lon, lat]
        },
        "properties": {
            'address': address,
            'name': name,
            'encode': encode,
            'city': city,
            'district': district,
            'town': town,
            'poiType': poiType

        }
    })

save_data = {"type": "FeatureCollection", "features": features}
with open(out_geojson_path, "w", encoding='utf-8') as f:
    f.write(json.dumps(save_data, ensure_ascii=False, indent=4, separators=(',', ':') ) )

In [ ]:
estate_info[['address','city','district','town','name','poiType','source','confidence','encode']].to_csv('/home/shengyuan/SH-COVID19/source/data/CaseInfo_April/estate_info_pub.csv')
print("estate_info_pub generated")